<a href="https://colab.research.google.com/github/donbcolab/composable_vlms/blob/main/notebooks/vision_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages (uncomment if needed)
!pip install -q pycocotools Pillow tqdm transformers torch datasets huggingface_hub wandb timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency

In [3]:
import os
from google.colab import userdata
import wandb

# setting the WANDB_API_KEY environment variable
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

In [4]:
import os
import logging
import requests
from typing import List, Dict, Any, Tuple
from pycocotools.coco import COCO
from datasets import load_dataset, Dataset
from huggingface_hub import HfApi, hf_hub_download
from tqdm import tqdm

# Configuration
CONFIG = {
    "annotation_file": '/content/annotations/instances_train2017.json',
    "image_directory": '/content/train2017/',
    "num_images": 500,
    "hf_dataset_name": "dwb2023/coco-subset-for-vision-evaluation",
    "output_directory": '/content/coco_subset/',
    "coco_annotations_url": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip",
    "coco_images_url": "http://images.cocodataset.org/zips/train2017.zip"
}

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



In [5]:
def download_file(url: str, save_path: str):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    with open(save_path, 'wb') as file, tqdm(
        desc=save_path,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as progress_bar:
        for data in response.iter_content(block_size):
            size = file.write(data)
            progress_bar.update(size)

def download_coco_data():
    import zipfile

    # Download annotations
    os.makedirs('/content/annotations', exist_ok=True)
    annotations_zip = '/content/annotations.zip'
    logging.info("Downloading COCO annotations...")
    download_file(CONFIG['coco_annotations_url'], annotations_zip)

    # Extract annotations
    logging.info("Extracting COCO annotations...")
    with zipfile.ZipFile(annotations_zip, 'r') as zip_ref:
        zip_ref.extractall('/content')
    os.remove(annotations_zip)

    # Download images
    os.makedirs(CONFIG['image_directory'], exist_ok=True)
    images_zip = '/content/train2017.zip'
    logging.info("Downloading COCO images...")
    download_file(CONFIG['coco_images_url'], images_zip)

    # Extract images
    logging.info("Extracting COCO images...")
    with zipfile.ZipFile(images_zip, 'r') as zip_ref:
        zip_ref.extractall('/content')
    os.remove(images_zip)

    logging.info("COCO data downloaded and extracted successfully")

In [6]:
def prepare_data() -> Tuple[List[Dict[str, Any]], COCO]:
    if not os.path.exists(CONFIG["annotation_file"]):
        logging.info("COCO annotations not found. Downloading COCO data...")
        download_coco_data()

    coco = COCO(CONFIG["annotation_file"])
    catIds = coco.getCatIds(catNms=['person', 'car'])
    imgIds = coco.getImgIds(catIds=catIds)
    images = coco.loadImgs(imgIds[:CONFIG["num_images"]])
    return images, coco

def prepare_and_upload_hf_dataset(images: List[Dict[str, Any]], coco: COCO, hf_dataset_name: str) -> Dataset:
    dataset_dict = {
        "image_id": [],
        "file_name": [],
        "width": [],
        "height": [],
        "annotations": []
    }

    for img in tqdm(images, desc="Preparing dataset"):
        dataset_dict["image_id"].append(img['id'])
        dataset_dict["file_name"].append(img['file_name'])
        dataset_dict["width"].append(img['width'])
        dataset_dict["height"].append(img['height'])
        dataset_dict["annotations"].append(coco.loadAnns(coco.getAnnIds(imgIds=img['id'])))

    dataset = Dataset.from_dict(dataset_dict)

    try:
        dataset.push_to_hub(CONFIG["hf_dataset_name"])
        logging.info(f"Dataset pushed to Hugging Face: {hf_dataset_name}")
    except Exception as e:
        logging.error(f"Failed to push dataset to Hugging Face: {e}")
        logging.info("Continuing without uploading to Hugging Face")

    return dataset

In [ ]:
def main():
    logging.info("Starting Vision Model Evaluation Setup")

    # Try to load existing dataset from Hugging Face
    try:
        dataset = load_dataset("imagefolder", data_dir=CONFIG["output_directory"])
        logging.info("Dataset created successfully using ImageFolder")
        images = dataset['train']
        coco = COCO(CONFIG["annotation_file"])
        logging.info(f"Loaded dataset from Hugging Face: {CONFIG['hf_dataset_name']}")
    except Exception as e:
        logging.warning(f"Failed to load Hugging Face dataset: {e}")
        logging.info("Starting data preparation")
        images, coco = prepare_data()
        dataset = prepare_and_upload_hf_dataset(images, coco, CONFIG["hf_dataset_name"])

    logging.info("Data preparation and loading complete")
    return dataset, coco

if __name__ == "__main__":
    dataset, coco = main()
    print("Dataset sample:", dataset[0])
    print("Number of images:", len(dataset))

/content/annotations.zip: 100%|██████████| 241M/241M [00:14<00:00, 17.6MiB/s]
/content/train2017.zip:  94%|█████████▎| 16.9G/18.0G [20:29<03:34, 5.79MiB/s]